# TensorRTに変換
Pytorchのresnet18モデルをTensorRTモデルに変換します。  

PytorchモデルをTensorRTモデルに変換するには、[torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) を利用します。  
モデルのベンチマーク速度は以下のように向上します。

|Model|Nano (PyTorch)|Nano (TensorRT)|Xavier (PyTorch)|Xavier (TensorRT)|
|-------------|-------------|-------------|-------------|-------------|
|alexnet|46.4|69.9|250|580|
|resnet18|29.4|90.2|140|712|

アプリケーション全体のFPSはモデル単体のベンチマーク結果よりも遅くなります。

In [ ]:
import torch
import torchvision

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()

学習済みモデルを読込みます。

In [ ]:
model.load_state_dict(torch.load('road_following_model.pth'))

TensorRTに変換します。

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

TernsorRTモデルを保存します。

In [ ]:
torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

## 次の作業

road_following.ipynb を実行し、TensorRTに変換した学習済みモデルで、実際の走行にトライします。